In [35]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [35]:
import psycopg

In [36]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from dotenv import load_dotenv
import os



In [37]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket_name = "d2b-internal-assessment-bucket"

# List all objects in the bucket
response = s3.list_objects(Bucket=bucket_name, Prefix="orders_data")

# Check if there are any objects in the response
if 'Contents' in response:
    # Loop through each object and print its key
    for obj in response['Contents']:
        print("File:", obj['Key'])

File: orders_data/
File: orders_data/analytics_export/faitusie8037/agg_public_holiday.csv
File: orders_data/analytics_export/faitusie8037/agg_shipments.csv
File: orders_data/analytics_export/faitusie8037/best_performing_product.csv
File: orders_data/analytics_export/murtodun9658/agg_public_holiday.csv
File: orders_data/analytics_export/murtodun9658/agg_shipments.csv
File: orders_data/analytics_export/murtodun9658/best_performing_product.csv
File: orders_data/analytics_export/murtodun9658/late_shipments.csv
File: orders_data/analytics_export/murtodun9658/undelivered_shipments.csv
File: orders_data/analytics_export/salimuti8986/agg_public_holiday.csv
File: orders_data/analytics_export/salimuti8986/agg_shipments.csv
File: orders_data/analytics_export/salimuti8986/best_performing_product.csv
File: orders_data/orders.csv
File: orders_data/reviews.csv
File: orders_data/shipment_deliveries.csv


In [38]:
# Download the orders.csv
s3.download_file(bucket_name, "orders_data/orders.csv", "orders.csv")

# Download the reviews.csv
s3.download_file(bucket_name, "orders_data/reviews.csv", "reviews.csv")

# Download the orders.csv
s3.download_file(bucket_name, "orders_data/shipment_deliveries.csv", "shipment_deliveries.csv")

In [57]:
# Read in fact_orders and format types

orders_types = {
    'order_id': int,
    'customer_id': int,
    'product_id': int,
    'unit_price': float,
    'quantity': int,
    'total_price': float
    
}


fact_orders = pd.read_csv('orders.csv', dtype=orders_types, parse_dates=['order_date'])


In [58]:
fact_orders.head()

,order_id,customer_id,order_date,product_id,unit_price,quantity,total_price
0,1,5,2022-07-13,24,139.0,10,1390.0
1,2,14,2021-04-06,2,273.0,4,1092.0
2,3,17,2022-07-29,20,253.0,9,2277.0
3,4,14,2022-08-27,8,334.0,1,334.0
4,5,25,2021-12-15,6,334.0,3,1002.0


In [60]:
# Read in fact_reviews and format types

reviews_types = {
    'review': int,
    'product_id': int
}

fact_reviews = pd.read_csv('reviews.csv', dtype=reviews_types)

In [62]:
fact_reviews.head()

,review,product_id
0,1,21
1,3,1
2,2,8
3,1,5
4,5,22


In [61]:
# Read in fact_shipment_deliveries and format types

shipments_types = {
    'shipment_id': int,
    'order_id': int
    
}


fact_shipment_deliveries = pd.read_csv('shipment_deliveries.csv', dtype=shipments_types, parse_dates=['shipment_date', 'delivery_date'])

In [63]:
fact_shipment_deliveries.head()

,shipment_id,order_id,shipment_date,delivery_date
0,1,1,2022-07-14,NaT
1,2,2,NaT,NaT
2,3,3,2022-07-31,2022-08-03
3,4,4,2022-09-02,2022-09-05
4,5,5,2021-12-19,2021-12-20


In [46]:
# Load environmental variables from the .env file
load_dotenv("C:\\Users\\USER\\Documents\\Data2Bots Assessment\\.env")


# Get the environmental variables
PG_USERNAME = os.getenv("PG_USERNAME")
PG_PASS = os.getenv("PG_PASS")
PG_HOST = os.getenv("PG_HOST")
PG_DB = os.getenv("PG_DB")

In [49]:
# Create PostgreSQL Connection
engine = create_engine(f'postgresql://{PG_USERNAME}:{PG_PASS}@{PG_HOST}:5432/{PG_DB}')

In [50]:
engine.connect()

In [66]:
print(pd.io.sql.get_schema(fact_orders, name='fact_orders', con=engine))


CREATE TABLE fact_orders (
	order_id INTEGER, 
	customer_id INTEGER, 
	order_date TIMESTAMP WITHOUT TIME ZONE, 
	product_id INTEGER, 
	unit_price FLOAT(53), 
	quantity INTEGER, 
	total_price FLOAT(53)
)




In [68]:
# Create fact_orders table in Database
fact_orders.head(n=0).to_sql(name='fact_orders', con=engine, schema='joshodey2178_staging', if_exists='replace')

1000

In [ ]:
# Load data into fact_orders table
fact_orders.to_sql(name='fact_orders', con=engine, schema='joshodey2178_staging', if_exists='append')

In [69]:
# Create fact_reviews table in Database
fact_reviews.head(n=0).to_sql(name='fact_reviews', con=engine, schema='joshodey2178_staging', if_exists='replace', index=False)

0

In [71]:
# Load data into fact_reviews table
fact_reviews.to_sql(name='fact_reviews', con=engine, schema='joshodey2178_staging', if_exists='append', index=False)

236

In [72]:
# Create fact_shipment_deliveries table
fact_shipment_deliveries.head(n=0).to_sql(name='fact_shipment_deliveries', con=engine, schema='joshodey2178_staging', if_exists='replace', index=False)


0

In [73]:
# Load data into fact_shipment_deliveries table
fact_shipment_deliveries.to_sql(name='fact_shipment_deliveries', con=engine, schema='joshodey2178_staging', if_exists='append', index=False)


1000